## Step 1:  Configuring the environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../../configure_and_validate.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).

## Step 2: Import the FABlib library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

User: choueiri@email.sc.edu bastion key is valid!
Configuration is valid


## Step 3: Create the experiment slice

The following creates three nodes with basic compute and networking capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

### Step 3.1: Create a slice
The code below creates a new slice with the name "csce585Project"

In [2]:
slice = fablib.new_slice(name="csce585Project")

### Step 3.2: Define the sites
The code below requests three random sites from FABRIC based on the condition that the following resources are available:

<ul>
    <li> 1 SmartNIC</li>
    <li> 8 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size
</ul>

In [5]:
sites= fablib.get_random_sites(count=1, filter_function=lambda x: x['nic_connectx_6_available'] > 2 and x['cores_available'] > 24 and x['ram_available'] > 48 and x['disk_available'] > 180)

print (f'The selected sites are {sites[0]}')

The selected sites are CERN


### Step 3.3: Creating the nodes
The code below creates three nodes: server1, server2 and server3. The servers use the following:
<ul>
    <li> 16 CPU cores</li>
    <li> 16GB RAM </li>
    <li> 200GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

server1 will be created in site1, server2 will be created in site2 and server3 will be created in site3

<img src="./figs/03_creating_nodes.png" width="550"><br>

In [5]:
server1 = slice.add_node(name="server1", 
                      site=sites[0], 
                      cores=8, 
                      ram=16, 
                      disk=60, 
                      image='default_ubuntu_20')

server2 = slice.add_node(name="server2", 
                      site=sites[0], 
                      cores=8, 
                      ram=16, 
                      disk=60, 
                      image='default_ubuntu_20')
server3 = slice.add_node(name="server3", 
                      site=sites[0], 
                      cores=8, 
                      ram=16, 
                      disk=60, 
                      image='default_ubuntu_20')

### Step 3.4: Adding the interfaces to the servers
The code below adds a Network Interface Card (NIC) to each server.

<img src="./figs/04_adding_nics.png" width="550"><br>

In [6]:
server1_iface = server1.add_component(model='NIC_ConnectX_6', name='nic').get_interfaces()[0]
server2_1_iface = server2.add_component(model='NIC_ConnectX_6', name='nic2').get_interfaces()[0]
server2_3_iface = server2.get_component(name='nic2').get_interfaces()[1]
server3_iface = server3.add_component(model='NIC_ConnectX_6', name='nic3').get_interfaces()[0]

### Step 3.5: Connecting server1 and server2
Create a network between server1 and server2 connecting them together and a network between server2 and server3 connecting them together.

<img src="./figs/05_connecting_nodes.png" width="550"><br>

In [7]:
net1 = slice.add_l2network(name='net1', interfaces=[server1_iface, server2_1_iface])
net2 = slice.add_l2network(name='net2', interfaces=[server2_3_iface, server3_iface])

### Step 3.6: Submitting the slice
The code below submits the slice. 
By default, the submit function will block until the node is ready and will display the progress of your slice being built.

In [8]:
slice.submit();


Retry: 11, Time: 268 sec


ID,cc0d6063-a5a5-4465-a775-3acd76e54d3d
Name,csce585Project
Lease Expiration (UTC),2025-11-27 00:03:51 +0000
Lease Start (UTC),2025-11-26 00:03:51 +0000
Project ID,7bd49888-1cce-4020-82e3-625d8b27f79f
State,StableOK
Email,CHOUEIRI@email.sc.edu
UserId,78504735-b34f-42a8-be20-8e71d6acf13e


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
9891ba35-41a4-4733-a659-6552083c8e94,server1,8,16,100,default_ubuntu_20,qcow2,cern-w4.fabric-testbed.net,CERN,ubuntu,2001:400:a100:3090:f816:3eff:fef5:7010,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fef5:7010,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
f46ed210-b13b-4bee-be1f-e23b40efa750,server2,8,16,100,default_ubuntu_20,qcow2,cern-w2.fabric-testbed.net,CERN,ubuntu,2001:400:a100:3090:f816:3eff:fe57:bf73,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fe57:bf73,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
09584529-27f1-42c2-a978-1ea79f8f1733,server3,8,16,100,default_ubuntu_20,qcow2,cern-w4.fabric-testbed.net,CERN,ubuntu,2001:400:a100:3090:f816:3eff:fe21:b6e6,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fe21:b6e6,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
4ad68a85-ef09-4a12-ba98-870a5b1634dc,net1,L2,L2Bridge,CERN,None,None,Active,
ef598b5c-5e9f-4a0a-92e5-5873a9a168a6,net2,L2,L2Bridge,CERN,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
server1-nic-p1,p1,server1,net1,100,config,,10:70:FD:E5:B6:60,enp7s0,enp7s0,fe80::1270:fdff:fee5:b660,6,HundredGigE0/0/0/17
server1-nic-p2,p2,server1,None,100,config,,10:70:FD:E5:B6:61,enp8s0,enp8s0,None,6,None
server2-nic2-p2,p2,server2,net2,100,config,,10:70:FD:E5:BD:D9,enp8s0,enp8s0,fe80::1270:fdff:fee5:bdd9,1,HundredGigE0/0/0/2
server2-nic2-p1,p1,server2,net1,100,config,,10:70:FD:E5:BD:D8,enp7s0,enp7s0,fe80::1270:fdff:fee5:bdd8,1,HundredGigE0/0/0/0
server3-nic3-p1,p1,server3,net2,100,config,,10:70:FD:E5:AB:D8,enp7s0,enp7s0,fe80::1270:fdff:fee5:abd8,1,HundredGigE0/0/0/16
server3-nic3-p2,p2,server3,None,100,config,,10:70:FD:E5:AB:D9,enp8s0,enp8s0,None,1,None



Time to print interfaces 278 seconds


## Step 4: Installing the required packages
In this step, we will install the required packages to run the lab. Specifically, we will install the Mellanox drivers, DPDK library, the P4 compiler (p4c), and all needed dependencies.

### Step 4.1 Appending list of servers
All servers are appended to a list to execute commands in parallel

In [10]:
slice = fablib.get_slice(name="csce585Project")

servers = []

servers.append(slice.get_node(name="server1"))     
servers.append(slice.get_node(name="server2"))
servers.append(slice.get_node(name="server3"))

server1 = servers[0]
server2 = servers[1]
server3 = servers[2]

### Step 4.2 NAT64 setup
The code below checks if an IPv6 address is available to set up NAT64. We will upload the script [scripts/nat64.sh](./scripts/nat64.sh) to the all servers and execute it

In [11]:
from ipaddress import ip_address, IPv6Address

threads = []

for server in servers:
    if type(ip_address(server.get_management_ip())) is IPv6Address:
        server.upload_file('scripts/nat64.sh', 'nat64.sh')
        threads.append(server.execute_thread(f'chmod +x nat64.sh && ./nat64.sh'))

for thread in threads:
    thread.result()

### Step 4.3 Installing dependencies
The code below installs packages that are prerequisites to the upcoming installations and needed to run the lab experiments

In [12]:
threads = []

for server in servers:
    threads.append(server.execute_thread('''
        sudo apt-get update;
        sudo apt-get install -y build-essential python3-pip python3-pyelftools libnuma-dev pkg-config net-tools hping3;
        sudo pip3 install meson ninja
    '''))

for thread in threads:
    thread.result()

### Step 4.4 Installing Mellanox drivers
Since ConnectX-6 NICs are used in this lab, it is essential to install the supporting drivers. The code below downloads and installs Mellanox drivers on all servers while enabling DPDK

In [13]:
threads = []

for server in servers:
    threads.append(server.execute_thread('''
        wget https://content.mellanox.com/ofed/MLNX_OFED-23.07-0.5.0.0/MLNX_OFED_LINUX-23.07-0.5.0.0-ubuntu20.04-x86_64.tgz; 
        tar xvfz MLNX_OFED_LINUX-23.07-0.5.0.0-ubuntu20.04-x86_64.tgz; 
        cd MLNX_OFED_LINUX-23.07-0.5.0.0-ubuntu20.04-x86_64; 
        echo "y" | sudo ./mlnxofedinstall --upstream-libs --dpdk --basic --without-fw-update --enable-sriov --hypervisor
    '''))
    
for thread in threads:
    thread.result()

### Step 4.5 Installing DPDK
The code below downloads, builds, and installs DPDK on all servers

In [ ]:
threads = []

for server in servers:
    threads.append(server.execute_thread('''
        git clone --branch v24.07-rc4 https://github.com/DPDK/dpdk.git; 
        cd dpdk;
        sudo meson build;
        cd build;
        sudo ninja;
        sudo ninja install; 
        sudo ldconfig
    '''))

for thread in threads:
    thread.result()

### Step 4.6 Installing Pktgen
In this lab, we will send from server1 packets at a high rate to server2 using a DPDK-based packet generation tool called pktgen [<a href="#References">5</a>]. The code below downloads and installs Pktgen-DPDK on all servers

In [ ]:
threads = []

for server in servers:
    threads.append(server.execute_thread('''
        sudo git clone --branch pktgen-24.07.1 https://github.com/pktgen/Pktgen-DPDK; 
        sudo sed -i \"s/deps += \\[dependency('numa', required: true)\\]/deps += \\[dependency('numa', required: false)\\]/\" /home/ubuntu/Pktgen-DPDK/app/meson.build;
        sudo apt-get install -y cmake libpcap-dev libbsd-dev;
        cd Pktgen-DPDK &&  sudo meson build && sudo ninja -C build && cd build/ && sudo meson install
    '''))

for thread in threads:
    thread.result()

### Step 4.7 Build pipeline library
The code below builds the DPDK pipeline library in server 2 (on which the pipeline will be running) to put all its functions into effect 

In [ ]:
stdout, stderr = server2.execute(f'cd dpdk/examples/pipeline && sudo make', quiet=True)

### Step 4.8 Install p4c
The code below downloads and installs the p4c compiler needed to compile the p4 code into a DPDK pipeline. In this lab, p4c is built from a version where the architecture has been modified.

In [ ]:
stdout, stderr = server2.execute('git clone https://github.com/CILab-USC/p4c.git', quiet = True)
stdout, stderr = server2.execute('sudo apt-get install -y cmake g++ git automake libtool libgc-dev bison flex libfl-dev libboost-dev libboost-iostreams-dev libboost-graph-dev llvm pkg-config python3 python3-pip tcpdump', quiet = True)
stdout, stderr = server2.execute('cd p4c && pip3 install --user -r requirements.txt && mkdir build && cd build && cmake .. && make -j4  && sudo make install', quiet = True)

### Step 4.9 Reboot
After installing the ConnectX Mellanox divers, it is essential to reboot the servers to complete the installation process or to ensure that updates are applied correctly

In [ ]:
for server in servers:
    server.os_reboot()

## Step 5: Configuring Network
In this step, we will assign IPv4 addresses to the interfaces of the servers and hardcode the MAC addresses. We will also configure forwarding and routing.

### Step 5.1: Get interfaces names
In this step we will get the interface names so that we can assign IP addresses to them. Map the printed interface names to those seen in this figure:

<img src="./figs/07_getting_interfaces.png" width="550"><br>

In [16]:
node1_iface = server1.get_interface(network_name='net1') 
server1_iface_name = node1_iface.get_device_name()+'np0'
print(f'server1_iface: {server1_iface_name}')

node2_iface1 = server2.get_interface(network_name='net1') 
server2_iface1_name = node2_iface1.get_device_name()+'np0'
print(f'server2_iface1: {server2_iface1_name}')

node2_iface2 = server2.get_interface(network_name='net2') 
server2_iface2_name = node2_iface2.get_device_name()+'np1'
print(f'server2_iface2: {server2_iface2_name}')

node3_iface = server3.get_interface(network_name='net2') 
server3_iface_name = node3_iface.get_device_name()+'np0'
print(f'server3_iface: {server3_iface_name}')

server1_iface: enp7s0np0
server2_iface1: enp7s0np0
server2_iface2: enp8s0np1
server3_iface: enp7s0np0


### Step 5.2: Turning all interfaces up
In this step, we will use the ip link command to turn the interfaces up

<img src="./figs/08_interfaces_up.png" width="550"><br>

In [17]:
stdout, stderr = server1.execute(f'sudo ip link set dev {server1_iface_name} up', quiet=True)
stdout, stderr = server2.execute(f'sudo ip link set dev {server2_iface1_name} up', quiet=True)
stdout, stderr = server2.execute(f'sudo ip link set dev {server2_iface2_name} up', quiet=True)
stdout, stderr = server3.execute(f'sudo ip link set dev {server3_iface_name} up', quiet=True)

### Step 5.3: Hardcode MAC addresses
For simplicity, we will use the following MAC addresses for the interfaces:
<ul>
    <li> server1_iface_MAC = '00:00:00:00:00:01' (shown as 00:01 in the figure below) </li>
    <li>server2_iface1_MAC = '00:00:00:00:00:21' (shown as 00:21 in the figure below)</li>
    <li>server2_iface2_MAC = '00:00:00:00:00:22' (shown as 00:22 in the figure below)</li>
    <li>server3_iface_MAC = '00:00:00:00:00:03' (shown as 00:03 in the figure below)</li>
</ul>

<img src="./figs/09_mac_addresses.png" width="550"><br>

In [18]:
server1_iface_MAC = '00:00:00:00:00:01'
server2_iface1_MAC = '00:00:00:00:00:21'
server2_iface2_MAC = '00:00:00:00:00:22'
server3_iface_MAC = '00:00:00:00:00:03'

### Step 5.4 Configuring the IP and MAC addresses on server1_iface and server2_iface1

We will use the network 192.168.10.0/24 between server1 and server2. We will assign the IP address 192.168.10.1 to server1's interface and 192.168.10.2 to its neighboring interface on server2.

<img src="./figs/10_IPs_1_2.png" width="550"><br>

In [19]:
server1_server2_subnet = "192.168.10.0/24"
server1_ip = '192.168.10.1/24'
server2_1_ip = '192.168.10.2/24'

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} {server1_ip}')
stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface1_name} {server2_1_ip}')

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} hw ether {server1_iface_MAC}')
stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface1_name} hw ether {server2_iface1_MAC}')

### Step 5.5 Configuring the IP and MAC addresses on server2_iface2 and server3_iface

We will use the network 192.168.20.0/24 between server2 and server3. We will assign the IP address 192.168.20.1 to server3's interface and 192.168.20.2 to its neighboring interface on server2.

<img src="./figs/11_IPs_2_3.png" width="550"><br>

In [20]:
server2_server3_subnet = "192.168.20.0/24"
server2_2_ip = '192.168.20.2/24'
server3_ip = '192.168.20.1/24'

stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface2_name} {server2_2_ip}')
stdout, stderr = server3.execute(f'sudo ifconfig {server3_iface_name} {server3_ip}')

stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface2_name} hw ether {server2_iface2_MAC}')
stdout, stderr = server3.execute(f'sudo ifconfig {server3_iface_name} hw ether {server3_iface_MAC}')

### Step 5.6: Enable forwarding on server2

The command "sudo sysctl -w net.ipv4.ip_forward=1" is used to enable IP forwarding on a Linux system.

IP forwarding is a feature that allows a system to act as a router by forwarding network packets from one network interface to another. By default, IP forwarding is usually disabled on Linux systems for security reasons. 

The command will be executed on the server2.

In [21]:
stdout, stderr = server2.execute(f'sudo sysctl -w net.ipv4.ip_forward=1', quiet=True)

### Step 5.7: Configure ARP and static routing

In this step, we will configure static ARP entries and static routing on server1, server2, and server3.

In [22]:
ip1 = server1_ip.split('/')[0]
ip2_1 = server2_1_ip.split('/')[0]
ip2_2 = server2_2_ip.split('/')[0]
ip3 = server3_ip.split('/')[0]
subnet1 = "192.168.10.0/24"
subnet2 = "192.168.20.0/24"

stdout, stderr = server1.execute(f'sudo arp -s {ip2_1} {server2_iface1_MAC}')
stdout, stderr = server1.execute(f'sudo ip route add {subnet2} via {ip2_1}')

stdout, stderr = server2.execute(f'sudo arp -s {ip1} {server1_iface_MAC}')
stdout, stderr = server2.execute(f'sudo arp -s {ip3} {server3_iface_MAC}')

stdout, stderr = server3.execute(f'sudo arp -s {ip2_2} {server2_iface2_MAC}')
stdout, stderr = server3.execute(f'sudo ip route add {subnet1} via {ip2_2}')

RTNETLINK answers: File exists
RTNETLINK answers: File exists


### Step 5.8: Mellanox devices

In this step, we will inspect and start all Mellanox devices.

In [23]:
stdout, stderr = server1.execute(f'sudo ibdev2netdev')
stdout, stderr = server1.execute(f'sudo mst status', quiet=True)
stdout, stderr = server1.execute(f'sudo mst start', quiet=True)
stdout, stderr = server1.execute(f'sudo mst status', quiet=True)

stdout, stderr = server2.execute(f'sudo ibdev2netdev')
stdout, stderr = server2.execute(f'sudo mst status', quiet=True)
stdout, stderr = server2.execute(f'sudo mst start', quiet=True)
stdout, stderr = server2.execute(f'sudo mst status', quiet=True)

stdout, stderr = server3.execute(f'sudo ibdev2netdev')
stdout, stderr = server3.execute(f'sudo mst status', quiet=True)
stdout, stderr = server3.execute(f'sudo mst start', quiet=True)
stdout, stderr = server3.execute(f'sudo mst status', quiet=True)

mlx5_0 port 1 ==> enp7s0np0 (Up)
mlx5_1 port 1 ==> enp8s0np1 (Down)
mlx5_0 port 1 ==> enp7s0np0 (Up)
mlx5_1 port 1 ==> enp8s0np1 (Up)
mlx5_0 port 1 ==> enp7s0np0 (Up)
mlx5_1 port 1 ==> enp8s0np1 (Down)


In the output above, you can see that there are two interfaces that are turned down. This is because each server has a Connect-X6 NIC attached, which is a dual-port NIC, and only one port in the NICs of server1 and server3 is used in this topology. 

## Step 6: Running the Application

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

slice = fablib.get_slice(name="itec760Project")

servers = []

servers.append(slice.get_node(name="server1"))     
servers.append(slice.get_node(name="server2"))
servers.append(slice.get_node(name="server3"))

server1 = servers[0]
server2 = servers[1]
server3 = servers[2]

User: choueiri@email.sc.edu bastion key is valid!
Configuration is valid


In [ ]:
server2.upload_file('scripts/code/classifier.p4','classifier.p4')
stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna classifier.p4 -o classifier.spec')
stdout, stderr = server2.execute(f'ls')

server2.upload_file('scripts/code/features_extract.p4','features_extract.p4')
stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna features_extract.p4 -o features_extract.spec')
stdout, stderr = server2.execute(f'ls')

In [11]:
server2.upload_file('scripts/code/pipeline.cli','pipeline.cli')
server2.upload_file('scripts/code/pipeline1.io','pipeline1.io')
server2.upload_file('scripts/code/pipeline2.io','pipeline2.io')

<SFTPAttributes: [ size=542 uid=1000 gid=1000 mode=0o100664 atime=1764121002 mtime=1764212631 ]>

In [12]:
server2.upload_file('scripts/rules/rules_code_table0.txt','rules_code_table0.txt')
server2.upload_file('scripts/rules/rules_code_table1.txt','rules_code_table1.txt')
server2.upload_file('scripts/rules/rules_code_table2.txt','rules_code_table2.txt')
server2.upload_file('scripts/rules/rules_iat_max.txt','rules_iat_max.txt')
server2.upload_file('scripts/rules/rules_iat_min.txt','rules_iat_min.txt')
server2.upload_file('scripts/rules/rules_max_differential_packet_length.txt','rules_max_differential_packet_length.txt')
server2.upload_file('scripts/rules/rules_min_differential_packet_length.txt','rules_min_differential_packet_length.txt')
server2.upload_file('scripts/rules/rules_packet_length_total.txt','rules_packet_length_total.txt')
server2.upload_file('scripts/rules/rules_voting_table.txt','rules_voting_table.txt')

<SFTPAttributes: [ size=488 uid=1000 gid=1000 mode=0o100664 atime=1764121002 mtime=1764212643 ]>

In [15]:
threads = []

for server in servers:
    threads.append(server.execute_thread(f' sudo sh -c  "echo 1024 > /sys/kernel/mm/hugepages/hugepages-2048kB/nr_hugepages"'))

for thread in threads:
    thread.result()

Hugepage reservation is done by setting the number of hugepages required to the ```nr_hugepages``` file in the kernel corresponding to a specific page size (in Kilobytes).

The ```echo``` command is used to print a value which in this case is ```1024``` representing the number of hugepages. The ```>``` symbol is a redirection operator that redirects the output of the previous command (echo 1024) to the file specified in the following path: ```/sys/kernel/mm/hugepages/hugepages-2048kB/nr_hugepages```

In [9]:
server2.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:103:a006:12:f816:3eff:fe94:95f6'

Run the following commands in the terminal:
    
    cd dpdk
    sudo examples/pipeline/build/pipeline -c 0x1F -- -s /home/ubuntu/pipeline.cli

# Step 7: Results

http://cicresearch.ca/IOTDataset/CIC_IOT_Dataset2023/Dataset/PCAP/Backdoor_Malware/

In [ ]:
stdout, stderr = server3.execute(f' wget http://cicresearch.ca/IOTDataset/CIC_IOT_Dataset2023/Dataset/PCAP/Benign_Final/BenignTraffic.pcap.pcap')
stdout, stderr = server3.execute(f' wget http://cicresearch.ca/IOTDataset/CIC_IOT_Dataset2023/Dataset/PCAP/DDoS-HTTP_Flood/DDoS-HTTP_Flood-.pcap')

In [5]:
server3.upload_file('scripts/data/BenignTraffic_test.pcap','BenignTraffic_test.pcap')
server3.upload_file('scripts/data/DDoS-HTTP_Flood-_test.pcap','DDoS-HTTP_Flood-_test.pcap')

<SFTPAttributes: [ size=53957552 uid=1000 gid=1000 mode=0o100664 atime=1764212156 mtime=1764212159 ]>

In [7]:
stdout, stderr = server3.execute(f' sudo tcprewrite --enet-smac=00:00:00:00:00:03 --enet-dmac=00:00:00:00:00:22 --infile=BenignTraffic_test.pcap --outfile=benign.pcap')

Warning in tcprewrite.c:post_args() line 222:
BenignTraffic_test.pcap was captured using a snaplen of 1500 bytes.  This may mean you have truncated packets.


In [8]:
stdout, stderr = server3.execute(f'sudo tcprewrite --enet-smac=00:00:00:00:00:03 --enet-dmac=00:00:00:00:00:22 --infile=DDoS-HTTP_Flood-_test.pcap --outfile=ddos.pcap')

Warning in tcprewrite.c:post_args() line 222:
DDoS-HTTP_Flood-_test.pcap was captured using a snaplen of 1500 bytes.  This may mean you have truncated packets.


In [18]:
server3.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:5e8:ff00:ffff:f816:3eff:fe31:e0d9'

In [ ]:
# sending packets from server 3 to the pipeline
# stdout, stderr = server3.execute(f'sudo tcpreplay -L 10 -i enp7s0np0 benign.pcap')
# stdout, stderr = server3.execute(f' sudo tcpreplay -L 10 -i enp7s0np0 ddos.pcap')
# stdout, stderr = server3.execute(f' sudo tcpdump -i enp7s0np0 -w out-benign,pcap')
# stdout, stderr = server3.execute(f' sudo tcpdump -i enp7s0np0 -w out-ddos,pcap')

In [ ]:
# inspecting classes in the registers of the pipeline
# telnet 0.0.0.0 8086
# pipeline> pipeline PIPELINE0 regrd reg_flow_index_0 index 0
# 0x7019
# pipeline> pipeline PIPELINE1 regrd reg_final_class_0 index 0x7019
# 0x1
# pipeline> pipeline PIPELINE0 regrd reg_flow_index_0 index 0
# 0x191
# pipeline> pipeline PIPELINE1 regrd reg_final_class_0 index 0x191
# 0x2

# telnet 0.0.0.0 8086


In [6]:
server3.download_file('out-benign.pcap','out-benign.pcap')
server3.download_file('out-ddos.pcap','out-ddos.pcap')